In [1]:
#package installs
#E:\Python310\python.exe -m pip install --upgrade pip
#!E:\Python310\python.exe -m pip install git+https://github.com/openai/whisper.git soundfile
!E:\Python310\Scripts\pip3.exe install ffmpeg av
#!E:\Python310\python.exe -m pip uninstall torch torchvision torchaudio -y
!E:\Python310\python.exe -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121


In [2]:
#Global variables
import socket
import os
ENV_HOSTNAME = socket.gethostname()
print('ENV_HOSTNAME:' + ENV_HOSTNAME)

#store defaults for Jacob here:
ENV_FOLDER_DATA = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data\\'
ENV_FOLDER_DATA_PROC = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data_Proc\\'
ENV_PRATT = ''


if ENV_HOSTNAME == 'JWGamingPC':
    ENV_FOLDER_DATA = 'E:\\W4732 Computer Vision\\Final Paper Data\\'
    ENV_FOLDER_DATA_PROC = 'E:\\W4732 Computer Vision\\Final Paper Data Proc\\'
    ENV_PRATT = ''

print('ENV_FOLDER_DATA:' + ENV_FOLDER_DATA)
print('ENV_PRATT:' + ENV_PRATT)

# Create folder structure
import os
os.makedirs(ENV_FOLDER_DATA_PROC + 'segmentation', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'targetdf', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'pratt', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'eps', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'clips', exist_ok=True)


ENV_HOSTNAME:JWGamingPC
ENV_FOLDER_DATA:E:\W4732 Computer Vision\Final Paper Data\
ENV_PRATT:


In [3]:
## Define Model here
# Need to define it up top to prevent overwriting it by mistake
import torch

class AudGenModel(torch.nn.Module):
    def __init__(self):
        super(AudGenModel, self).__init__()

        chanMult = 56
        kernel = 4
        stride = 2
        num_frames = 40
        self.relu = torch.nn.LeakyReLU(0.2, inplace=True)
        self.sigmoid = torch.nn.Sigmoid()

        self.conv1 = torch.nn.Conv3d(in_channels = 1, out_channels = chanMult
                                     , kernel_size = (1,360,640) ,stride = stride ,padding = 1 
                                     #, depth=num_frames
                                    )
        self.conv2 = torch.nn.Conv3d(in_channels = chanMult, out_channels = 2 * chanMult
                                     , kernel_size = kernel ,stride = stride ,padding = 1
                                     #, depth=num_frames
                                     )
        self.conv3 = torch.nn.Conv3d(in_channels = 2 * chanMult, out_channels = 4 * chanMult
                                     , kernel_size = 3 ,stride = stride ,padding = 1
                                     #, depth=num_frames
                                     )
        self.output = torch.nn.Linear(1120, 41)


    def forward(self, input):


        ################################
        # Please fill in your code here:
        ################################
        
        x = self.relu(self.conv1(input))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = torch.flatten(x,0) #keep the 1st dim since it's the batch
        #x = self.relu(self.hidden(x))
        #x = torch.functional.dropout(x,0.25)
        x = self.output(x)
        out = self.sigmoid(x)

        
        return out
    
a = AudGenModel()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
a = a.to(device)

In [6]:
## Model Training 1 ##
import pandas as pd
import json
import copy
import csv
import pickle
import torchvision
import itertools
import torch
import glob
import random



#get number after # but before space afterwards

#generate filepaths
#filepath_mp4 = ENV_FOLDER_DATA + filesuf_mp4
#folderpath_normmp4 = ENV_FOLDER_DATA_PROC + 'norm\\' + str_epnum + '\\'
#os.makedirs(folderpath_normmp4, exist_ok=True)
#filepath_normmp4 = folderpath_normmp4 + str_epnum + '.mp4'
#folderpath_eps = ENV_FOLDER_DATA_PROC + 'eps\\' + str_epnum + '\\'
#os.makedirs(folderpath_eps, exist_ok=True)
#folderpath_clips = ENV_FOLDER_DATA_PROC + 'clips\\' + str_epnum + '\\'
#os.makedirs(folderpath_clips, exist_ok=True)
folderpath_clipinfo = ENV_FOLDER_DATA_PROC + 'clipinfo\\'
os.makedirs(folderpath_clipinfo, exist_ok=True)
folderpath_sourcedf = ENV_FOLDER_DATA_PROC + 'sourcedf\\'
os.makedirs(folderpath_sourcedf, exist_ok=True)
#filepath_sourcedf = folderpath_sourcedf + str_epnum + '.pickle'
#folderpath_wavsplit = folderpath_eps + 'wavsplit\\'
#os.makedirs(folderpath_wavsplit, exist_ok=True)
#filepath_segmentation = ENV_FOLDER_DATA_PROC + 'segmentation\\' + str_epnum + '.psv'
folderpath_targetdf = ENV_FOLDER_DATA_PROC + 'targetdf\\'
folderpath_pratt = ENV_FOLDER_DATA_PROC + 'pratt\\' 
filepath_speakers = ENV_FOLDER_DATA_PROC + 'speakers.json'

## get overlap between sourcedf files and targetdf files
list_eps = []

#note - source is actually a dict of dict of tensors
dict_sourcedf = {}
list_sourcedf_eps = []
for path_pickle in glob.glob(folderpath_sourcedf + '*.pickle'):
    dict_temp = {}
    dict_temp['path'] = path_pickle
    str_basenm = os.path.basename(path_pickle)
    dict_temp['basenm'] = str_basenm
    dict_temp['str_epnum'] = str_basenm.split('.')[0]
    int_epnum = int(dict_temp['str_epnum'])
    dict_temp['int_epnum'] = int_epnum
    list_sourcedf_eps.append(int_epnum)
    dict_sourcedf[int_epnum] = dict_temp

#note - target is actually a df
dict_targetdf = {}
list_targetdf_eps = []
for path_pickle in glob.glob(folderpath_targetdf + '*.pickle'):
    dict_temp = {}
    dict_temp['path'] = path_pickle
    str_basenm = os.path.basename(path_pickle)
    dict_temp['basenm'] = str_basenm
    dict_temp['str_epnum'] = str_basenm.split('.')[0]
    int_epnum = int(dict_temp['str_epnum'])
    dict_temp['int_epnum'] = int_epnum
    list_targetdf_eps.append(int_epnum)
    dict_targetdf[int_epnum] = dict_temp

#populate matches
for src_ep in list_sourcedf_eps:
    if src_ep in list_targetdf_eps:
        list_eps.append(src_ep)

#40 data elements
list_datacols = []
list_datacols.append('m')
for i in range(40):
    list_datacols.append('d' + str(i))

#define model pieces
NUM_EPOCHS = 50
TRAINING_TYPE = 'p'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform=torchvision.transforms.Normalize( mean=(255*0.5), std=(255*0.5) )
optimizer = torch.optim.Adam(a.parameters(), lr=0.0008, betas=(0.65, 0.999))
criterion = torch.nn.MSELoss().to(device)


##Epoch Loop
for e in range(NUM_EPOCHS):
    #make a copy of the episode list to pop out 2 at a time
    list_epocheps = copy.deepcopy(list_eps)
    while len(list_epocheps) > 0:
        #load two or 3 episodes into memory at a time
        list_tempeps = []
        for i in range(2):
            if len(list_epocheps) > 0:
                list_tempeps.append(list_epocheps.pop())
        #load pratt files into memory
        dict_target = {}
        for i,tempeps in enumerate(list_tempeps):
            with open(dict_targetdf[tempeps]['path'], 'rb') as file:
                dict_target[tempeps] =  pickle.load(file)
                    
        #load video files into memory
        dict_source = {}
        for i,tempeps in enumerate(list_tempeps):
            with open(dict_sourcedf[tempeps]['path'], 'rb') as file:
                dict_temp = pickle.load(file)
                dict_source[tempeps] = dict_temp

        #create a list of eps / clips and shuffle
        listtup_epseg = []
        for key,val in dict_source.items():
            for key2,val2 in val.items():
                tup_epseg = [key,key2]
                listtup_epseg.append( tup_epseg)
        random.shuffle(tup_epseg)

        #loop for data elements
        for tup_epseg in listtup_epseg:
            int_epnum = tup_epseg[0]
            int_segnum = tup_epseg[1]
            #feed values into the predict / loss / optimize cycle
            inputs = None
            labels = None
            if int_epnum not in dict_source.keys():
                continue
            if int_segnum not in dict_source[int_epnum].keys():
                continue
            inputs = dict_source[int_epnum][int_segnum]
            #pad inputs
            #for x in range(40 - len()):
            #    inputs.append( torch.empty((1,360,640), dtype=torch.int8) )
            #inputs = torch.stack(inputs[:40], 0)
            #transform inputs
            #print(inputs.size())
            #torch.Size([40, 1, 360, 640])
            inputs = transform(inputs.to(torch.float)).movedim(0,1)
            #print(inputs.size())
            

            #check if labels exist
            label_df = dict_target[int_epnum].query('seg==' + str(int_segnum) + ' & type=="' + TRAINING_TYPE + '"' )
            if len(label_df) == 0:
                print('No labels for ep:' + str(int_epnum) + ' seg:' + str(int_segnum))
                continue
            labels = torch.squeeze(torch.tensor(label_df[list_datacols].fillna(0).values).to(torch.float))
            #normalize labels 
            if TRAINING_TYPE == 'p':
                labels = (labels-150) / 150.0
            elif TRAINING_TYPE == 'i':
                labels = (labels - 80) / 80.0
            elif TRAINING_TYPE == 's':
                labels = labels
            elif TRAINING_TYPE == 'j':
                labels = labels
            elif TRAINING_TYPE == 'h':
                labels = (labels - 10) / 10
            

            inputs, labels = inputs.to(device), labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = a(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # print statistics
        #if i % 1000 == 0:
        #    print(f'Epoch={epoch + 1} Iter={i + 1:5d} Loss={loss.item():.3f}')
        #    running_loss = 0.0




    #define a function that, given a targetdf value, look up the source - if both exists, use the data to train the model

#end loop through data

KeyboardInterrupt: 

In [ ]:
#save model down for future use / training
